In [8]:
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from kloppy.domain.models import Point
from typing import Sequence, Union

In [2]:
from sklearn.model_selection import train_test_split
from soccer_analytics.data.statsbomb import load_competition_seasons
from soccer_analytics.data_split import split_by_time
from soccer_analytics.preprocessing import match_list_to_df
matches = load_competition_seasons(
    "FIFA World Cup", ["2022"], event_types=["shot"]
)
train_test_matches, _ = split_by_time(matches, test_frac=0.2)
train_matches, test_matches = train_test_split(
    train_test_matches, test_size=0.2, random_state=235
)
len(train_matches)

40

In [7]:
df = match_list_to_df(train_matches)
df.freeze_frame.iloc[0]

[{'location': [105.7, 38.8],
  'player': {'id': 33572, 'name': 'Nathaniel Atkinson'},
  'position': {'id': 2, 'name': 'Right Back'},
  'teammate': False},
 {'location': [107.8, 32.9],
  'player': {'id': 5481, 'name': 'Mathew Leckie'},
  'position': {'id': 12, 'name': 'Right Midfield'},
  'teammate': False},
 {'location': [107.1, 31.2],
  'player': {'id': 33477, 'name': 'Riley McGree'},
  'position': {'id': 15, 'name': 'Left Center Midfield'},
  'teammate': False},
 {'location': [98.2, 46.6],
  'player': {'id': 8346, 'name': 'Craig Goodwin'},
  'position': {'id': 16, 'name': 'Left Midfield'},
  'teammate': False},
 {'location': [113.4, 36.0],
  'player': {'id': 5490, 'name': 'Jackson Irvine'},
  'position': {'id': 13, 'name': 'Right Center Midfield'},
  'teammate': False},
 {'location': [109.4, 35.6],
  'player': {'id': 22293, 'name': 'Harry Souttar'},
  'position': {'id': 3, 'name': 'Right Center Back'},
  'teammate': False},
 {'location': [109.9, 41.6],
  'player': {'id': 33495, 'name

In [ ]:
"""
For shot on goal:
  1. Get all potential blockers
  2. Generate y values linspaced across the goalmouth
  3. For each potential blocker:
     a. Get function that takes in goal position and maps to block %
       -- can take in player data too, e.g. set all teammate block % lower
     b. execute function across the y values, add to running list
  4. Collapse list down to overall value
  5. return overall value
  
"""

In [ ]:
def map_goal_locations_to_distance_from_blocker(
        shot_location: Point, 
        blocker_location: Point, 
        goal_x: float, 
        goal_y_values: np.array
):
    """
    Given an array of points along a goalmouth (or the entire end-line, really),
    Identify how close to the line drawn between the shot location and the blocker
    location is to the line between the shot location and each point on the goalmouth.
    
    The minimum distance occurs at the point perpendicular to the line drawn between the shot
    location and the blocker location, so this problem collapses to identifying that distance.
    """
    # First, if the blocker is not between the shot and the goal, return a null value:
    if blocker_location.x <= shot_location.x:
        return np.full_like(goal_y_values, np.nan)
    # Assuming the blocker is in a position to potentiall impact the play, 
    # get the angle between the shot and the blocker compared to the horizontal axis:
    shot_blocker_angle = math.atan2(
        blocker_location.y - shot_location.y, 
        blocker_location.x - shot_location.x
    )
    # The distance between the shot and the blocker
    shot_blocker_distance = math.sqrt(
        (blocker_location.y - shot_location.y) ** 2
        + (blocker_location.x - shot_location.x) ** 2
    )
    # Get the differences in y and x values between the shot location and the goalline:
    x_diff = goal_x - shot_location.x
    y_diff = goal_y_values - shot_location.y
    
    # The min distance is the tangent between the incremental angle of the y_value on the
    # goalline above the shot/blocker angle, multiplied by the distance between the shot and
    # the blocker:
    min_distances = shot_blocker_distance * math.tan(math.atan2(y_diff, x_diff) - shot_blocker_angle)
    return min_distances
    

In [ ]:
def get_block_likelihood(
        shot_location: Point, 
        blocker_info: list[dict[str, Union[list, dict, str]]], 
        block_function, 
        goal_x=120., 
        goal_ys=(36, 44), 
        num_test_points=1000
    ):
    goal_y_values = np.linspace(*goal_ys, num_test_points)
    y_block_likelihoods_per_player = np.zeros((test_y_values, len(blocker_info)))
    for i, blocker in enumerate(blocker_info):
        blocker_distances = map_goal_locations_to_distance_from_blocker(
            shot_location, blocker["location"], goal_x, goal_y_values
        )
        y_block_likelihoods_per_player[:, i] = block_function(blocker_distances, shot_location, blocker, goal_x)
        
    no_block_likelihoods_per_player = 1 - y_block_likelihoods_per_player
    no_block_likelihoods = np.multiply.reduce(
        no_block_likelihoods_per_player,
        axis=1
    )
    return 1 - no_block_likelihoods.mean()
    

In [ ]:
def map_block_locations(
        shot_location: Point, blocker_location: Point, offset: float,
        goal_x: float = 120.
):
    shot_blocker_angle = math.atan2(
        blocker_location.y - shot_location.y, 
        blocker_location.x - shot_location.x
    )
    offset_x = -offset * math.sin(shot_blocker_angle)
    offset_y = offset * math.cos(shot_blocker_angle)
    offset_angle = math.atan2(
        offset_y - shot_location.y,
        offset_x - shot_location.x
    )
    offset_y_at_goal = shot_location.y + goal_x * math.tan(offset_angle)
    return offset_y_at_goal
    

In [ ]:
test = np.arange(10).reshape(5, -1)
test

In [ ]:
np.multiply.reduce(test, axis=1).mean()